In [4]:
# coding: utf-8

# In[3]:


from __future__ import print_function

import sys
import numpy as np
from time import time
import matplotlib.pyplot as plt 

sys.path.append('/home/xilinx')
from pynq import Overlay
from pynq import allocate
from scipy import misc

if __name__ == "__main__":
    print("Entry:", sys.argv[0])
    print("System argument(s):", len(sys.argv))

    print("Start of \"" + sys.argv[0] + "\"")

    ol = Overlay("./design_2_hist.bit")
    ipModule = ol.wrapper_img_hist_equ_0
    ipDMAIn0 = ol.axi_dma_0
    ipDMAOut0 = ol.axi_dma_1
    
    
    # read data
    image= misc.imread('./data/test_1080p.bmp', flatten= 0)
    IFlat = image.flatten()
    k = 256*257+1
    k2 = 256*256
    inBuffer0 = allocate(shape=(1080*1920,), dtype=np.int32)

    
    
    
    for i in range(1080*1920):
        inBuffer0[i]= IFlat[0+i*3]*(k2) + IFlat[1+i*3]*(256) + IFlat[2+i*3]
        #inBuffer0[i]= IFlat[1+i*3]*(k)

        
    
    
    #initialization
    outBuffer0 = allocate(shape=(1080*1920,), dtype=np.int32)
    outcdf = allocate(shape=(256,), dtype=np.int32)
    inhist = allocate(shape=(256,), dtype=np.int32)
    

    
    print('FPGA result')
    
    timeKernelStart = time()
    ipModule.write(0x010, 1080)
    ipModule.write(0x018, 1920)
    for i in range(256):
        ipModule.write(1024+i*4, int(outcdf[i]))
        ipModule.write(2048+i*4, int(inhist[i]))
        #print(i)
    
    ipModule.write(0x00, 0x01)
        
    ipDMAIn0.sendchannel.transfer(inBuffer0)
    ipDMAOut0.recvchannel.transfer(outBuffer0)
    ipDMAIn0.sendchannel.wait()
    ipDMAOut0.recvchannel.wait()
    timeKernelEnd = time()
    print("Kernel execution time: " + str(timeKernelEnd - timeKernelStart) + " s")
    
    
    
    for i in range(256):
        outcdf[i] = ipModule.read(1024+i*4)
        inhist[i] = ipModule.read(2048+i*4)
        
    with open('test.npy', 'wb') as f:

        np.save(f, outBuffer0)
        
    with open('inhist.npy', 'wb') as f:

        np.save(f, inhist)          
        
    with open('outcdf.npy', 'wb') as f:

        np.save(f, outcdf)   
    
    print("============================")
    print("Exit process")

Entry: /usr/lib/python3/dist-packages/ipykernel_launcher.py
System argument(s): 3
Start of "/usr/lib/python3/dist-packages/ipykernel_launcher.py"
[22 27 33]
FPGA result
Kernel execution time: 0.0435640811920166 s
Exit process
